In [1]:
import pandas as pd

In [2]:
channel1 = 'Scikitlearn'
channel2 = 'Angular'

## Respondent Analysis

### Scikitlearn

In [21]:
df_scikit_profile = pd.read_csv(channel1+'/'+channel1+'_profile.csv')

In [22]:
df_scikit_profile.head()

,user_Id,cluster_Id,answer_date,answers
0,54d4a1d6db8155e6700f853b,12,2015-02-06 11:14:00,I'm with family this week and so not super act...
1,54d4a1d6db8155e6700f853b,61,2015-02-24 22:46:00,cool\r\nok\r\n
2,54d4a1d6db8155e6700f853b,69,2015-02-25 18:59:00,yeah that sounds like a good idea\r\n
3,54d4a1d6db8155e6700f853b,72,2015-02-25 19:14:00,you mean == 1\r\n
4,54d4a1d6db8155e6700f853b,75,2015-02-25 19:19:00,that is also weird ;)\r\n


In [23]:
df_scikit_sorted = df_scikit_profile.groupby(df_scikit_profile.user_Id)["user_Id"].count().reset_index(name='count').sort_values(['count'], ascending=False,ignore_index= True)

In [24]:
df_scikit_sorted.describe()

,count
count,106.000000
mean,5.773585
std,19.789503
min,1.000000
25%,1.000000
50%,1.000000
75%,3.000000
max,186.000000


In [25]:
len(df_scikit_sorted[df_scikit_sorted['count'] >= 10])

9

In [26]:
len(df_scikit_sorted[df_scikit_sorted['count'] >= 100])

1

In [28]:
print("dialogues with response:",len(set(df_scikit_profile["cluster_Id"])))

dialogues with response: 595


In [52]:
df_scikit_issue = pd.read_json(channel1+'/'+channel1+'_issue.json')

In [53]:
df_scikit_issue.head()

,Id,asker,time,content,lines
0,1,541a528b163965c9bc2053de,2014-09-18 03:36,Hi all!\n,1
1,2,544906e2db8155e6700cdd16,2014-10-23 19:59,I don't know if this is the right place but I ...,2
2,3,544906e2db8155e6700cdd16,2014-10-23 20:01,My first approach was to convert each feature ...,2
3,4,53a5cf04a9176b500d1ced1a,2015-01-23 15:38,This message is without content. I wonder if a...,1
4,5,541a528b163965c9bc2053de,2015-01-23 16:49,"@mac2bua, I just saw your message. It's probab...",1


### Angular

In [86]:
df_angular_profile = pd.read_csv(channel2+'/'+channel2+'_profile.csv')

In [87]:
df_angular_profile.head()

,user_Id,cluster_Id,answer_date,answers
0,54e6ee3b15522ed4b3dc3c24,15,2015-03-08 18:01:00,currently only working in chrome until shadow ...
1,54e6ee3b15522ed4b3dc3c24,20,2015-03-09 02:39:00,::shadow\nhttp://www.html5rocks.com/en/tutoria...
2,54e6ee3b15522ed4b3dc3c24,100,2015-03-13 19:48:00,@mhevery that was my worry too (Dart version ...
3,54e6ee3b15522ed4b3dc3c24,113,2015-03-14 02:46:00,you donat need to inject it since itas going t...
4,54e6ee3b15522ed4b3dc3c24,118,2015-03-14 07:23:00,what version of node?\n


In [88]:
df_angular_sorted = df_angular_profile.groupby(df_angular_profile.user_Id)["user_Id"].count().reset_index(name='count').sort_values(['count'], ascending=False,ignore_index= True)

In [41]:
df_angular_sorted.describe()

,count
count,6707.000000
mean,21.891904
std,232.142550
min,1.000000
25%,1.000000
50%,3.000000
75%,7.000000
max,11700.000000


In [7]:
len(df_angular_sorted[df_angular_sorted['count'] >= 10])

1317

In [8]:
len(df_angular_sorted[df_angular_sorted['count'] >= 100])

163

In [9]:
print("dialogues with response:",len(set(df_angular_profile["cluster_Id"])))

dialogues with response: 122791


In [10]:
df_angular_issue = pd.read_json(channel2+'/'+channel2+'_issue.json')

In [11]:
df_angular_issue.head()

,Id,asker,time,content,lines
0,1,54c89b1cdb8155e6700f2ac3,2015-03-07 21:41,Awesome!\n,1
1,2,54e6ee3b15522ed4b3dc3c24,2015-03-07 21:53,"sweet, now we can ask angular2 questions in re...",1
2,3,530cd1eb5e986b0712efb4d2,2015-03-07 22:04,Yes!\n,1
3,4,54e6ee3b15522ed4b3dc3c24,2015-03-07 22:06,we're missing the [View Source] [Improve this ...,1
4,5,530cd1eb5e986b0712efb4d2,2015-03-07 23:18,Good idea can you file an issue with repo?\n,1


## Tags Matching

In [14]:
import nltk
from nltk.corpus import stopwords

In [15]:
def tags_match(df,tags):
    profile_tags = []
    total_tags = set()
    for answer in df:
        tags_matched = []
    
        if len(str(answer)) == 0:
            profile_tags.append(tags_matched)
            continue
    
        for tag in tags:
            
            if str(tag) in str(answer):
                tags_matched.append(tag)
                total_tags.add(tag)
    
        profile_tags.append(tags_matched)
        
    print("total tags matched:",len(total_tags))
    
    return profile_tags,total_tags

In [16]:
df_tags = pd.read_csv('tags.csv')

In [17]:
len(df_tags)

63581

In [18]:
df_tags.head()

,Id,TagName,Count,ExcerptPostId,WikiPostId,IsModeratorOnly,IsRequired
0,1,.net,322128,3624959.0,3607476.0,NaN,NaN
1,2,html,1141471,3673183.0,3673182.0,NaN,NaN
2,3,javascript,2413015,3624960.0,3607052.0,NaN,NaN
3,4,css,768321,3644670.0,3644669.0,NaN,NaN
4,5,php,1443156,3624936.0,3607050.0,NaN,NaN


In [19]:
tags_clean = set()
for tag in df_tags["TagName"]:
    if len(str(tag)) > 1 and str(tag) not in stopwords.words('english'):
        tags_clean.add(str(tag))

In [20]:
len(tags_clean)

63550

### Scikitlearn

In [59]:
scikit_tags,scikit_matched = tags_match(df_scikit_profile["answers"],tags_clean)

total tags matched: 1264


In [48]:
df_scikit_profile["tags"] = scikit_tags

In [49]:
df_scikit_profile.head()

,user_Id,cluster_Id,answer_date,answers,tags
0,54d4a1d6db8155e6700f853b,12,2015-02-06 11:14:00,I'm with family this week and so not super act...,"[import, tps, dtype, ase, el, hub, dt, om, nt,..."
1,54d4a1d6db8155e6700f853b,61,2015-02-24 22:46:00,cool\r\nok\r\n,[co]
2,54d4a1d6db8155e6700f853b,69,2015-02-25 18:59:00,yeah that sounds like a good idea\r\n,"[de, goo, ide, hat, go, ou, od]"
3,54d4a1d6db8155e6700f853b,72,2015-02-25 19:14:00,you mean == 1\r\n,"[mean, yo, ou]"
4,54d4a1d6db8155e6700f853b,75,2015-02-25 19:19:00,that is also weird ;)\r\n,"[ei, rd, hat, lso, als, ls]"


In [51]:
df_scikit_profile.to_csv(channel1+'/'+channel1+'_profile_tags.csv',index = None)

In [54]:
scikit_issue_tags,scikit_issue_matched = tags_match(df_scikit_issue["content"],tags_clean)

total tags matched: 3632


In [55]:
df_scikit_issue["tags"] = scikit_issue_tags

In [56]:
df_scikit_issue.head()

,Id,asker,time,content,lines,tags
0,1,541a528b163965c9bc2053de,2014-09-18 03:36,Hi all!\n,1,[]
1,2,544906e2db8155e6700cdd16,2014-10-23 19:59,I don't know if this is the right place but I ...,2,"[int, dvi, set, som, tr, ip, ri, om, nt, de, c..."
2,3,544906e2db8155e6700cdd16,2014-10-23 20:01,My first approach was to convert each feature ...,2,"[el, alu, rise, ri, bin, nt, val, ex, live, am..."
3,4,53a5cf04a9176b500d1ced1a,2015-01-23 15:38,This message is without content. I wonder if a...,1,"[der, ag, nt, de, ess, sage, co, yo, ent, sag,..."
4,5,541a528b163965c9bc2053de,2015-01-23 16:49,"@mac2bua, I just saw your message. It's probab...",1,"[int, el, ag, jus, sly, quest, overflow, tr, o..."


In [57]:
df_scikit_issue.to_csv(channel1+'/'+channel1+'_issue_tags.csv',index = None)

In [60]:
len(scikit_matched)

1264

In [61]:
clusters_answered = set(df_scikit_profile["cluster_Id"])

In [62]:
len(clusters_answered)

595

In [63]:
df_scikit_issue_selected = df_scikit_issue[df_scikit_issue["Id"].isin(clusters_answered)]

In [64]:
len(df_scikit_issue_selected)

595

In [69]:
df_scikit_issue_selected.head()

,Id,asker,time,content,lines,tags
11,12,541a528b163965c9bc2053de,2015-02-06 11:14,I want to do the `check_array` empty data first\n,9,"[pty, nt, ant, ta, wan, ata, st, ch, arr, ray,..."
12,13,54d4a1d6db8155e6700f853b,2015-02-06 11:18,Most of the bug-fixes are isolated to one esti...,9,"[int, import, tps, dtype, ase, el, pipeline, i..."
13,14,54d4a1d6db8155e6700f853b,2015-02-06 11:23,it did? I didn't see that. haha I know the ope...,8,"[sis, ase, sf, el, som, ini, tr, rm, ode, ip, ..."
37,38,541a528b163965c9bc2053de,2015-02-15 16:33,Explaining the pbm in an issue is a good idea....,2,"[opl, unc, el, ini, de, bunch, goo, ch, rel, i..."
51,52,541a528b163965c9bc2053de,2015-02-24 14:06,cython path/to/file.pyx\n,2,"[cython, path, pyx, file, il, yt]"


In [81]:
issue_matched = set()
for tags in df_scikit_issue_selected["tags"]:
    issue_matched.update(tags)

In [83]:
len(issue_matched)

1741

In [85]:
print("final matched:",len(scikit_matched.union(issue_matched)))

final matched: 1977


### Angular

In [21]:
angular_tags,angular_matched = tags_match(df_angular_profile["answers"],tags_clean)

total tags matched: 7637


In [92]:
#save angular_matched
with open(channel2+'/'+channel2+'_angular_matched.txt', 'w') as fp:
    for name in angular_matched:
        # write each item on a new line
        fp.write("%s\n" % name)
    print('Done')

Done


In [25]:
df_angular_profile["tags"] = angular_tags

In [26]:
df_angular_profile.head()

,user_Id,cluster_Id,answer_date,answers,tags
0,54e6ee3b15522ed4b3dc3c24,15,2015-03-08 18:01:00,currently only working in chrome until shadow ...,"[rr, urn, ent, em, kin, dom, ado, default, ntl..."
1,54e6ee3b15522ed4b3dc3c24,20,2015-03-09 02:39:00,::shadow\nhttp://www.html5rocks.com/en/tutoria...,"[styling, co, ld, parent, tor, ria, ri, ent, r..."
2,54e6ee3b15522ed4b3dc3c24,100,2015-03-13 19:48:00,@mhevery that was my worry too (Dart version ...,"[rr, cad, urn, wil, co, ld, dependencies, tera..."
3,54e6ee3b15522ed4b3dc3c24,113,2015-03-14 02:46:00,you donat need to inject it since itas going t...,"[ss, inline, int, nat, co, ld, tor, ent, em, s..."
4,54e6ee3b15522ed4b3dc3c24,118,2015-03-14 07:23:00,what version of node?\n,"[rs, hat, io, de, od, ode, version, rsi]"


In [27]:
df_angular_profile.to_csv(channel2+'/'+channel2+'_profile_tags.csv',index = None)

In [28]:
angular_issue_tags,angular_issue_matched = tags_match(df_angular_issue["content"],tags_clean)

total tags matched: 12189


In [29]:
df_angular_issue["tags"] = angular_issue_tags

In [30]:
df_angular_issue.head()

,Id,asker,time,content,lines,tags
0,1,54c89b1cdb8155e6700f2ac3,2015-03-07 21:41,Awesome!\n,1,"[som, om]"
1,2,54e6ee3b15522ed4b3dc3c24,2015-03-07 21:53,"sweet, now we can ask angular2 questions in re...",1,"[time, sti, io, angular, ime, real-time, st, q..."
2,3,530cd1eb5e986b0712efb4d2,2015-03-07 22:04,Yes!\n,1,[]
3,4,54e6ee3b15522ed4b3dc3c24,2015-03-07 22:06,we're missing the [View Source] [Improve this ...,1,"[ss, ton, angular.io, rce, ou, io, angular, ss..."
4,5,530cd1eb5e986b0712efb4d2,2015-03-07 23:18,Good idea can you file an issue with repo?\n,1,"[ss, repo, su, ou, rep, il, yo, de, od, ide, p..."


In [31]:
df_angular_issue.to_csv(channel2+'/'+channel2+'_issue_tags.csv',index = None)

In [32]:
clusters_answered = set(df_angular_profile["cluster_Id"])

In [33]:
len(clusters_answered)

122791

In [34]:
df_angular_issue_selected = df_angular_issue[df_angular_issue["Id"].isin(clusters_answered)]

In [35]:
len(df_angular_issue_selected)

122791

In [36]:
issue_matched = set()
for tags in df_angular_issue_selected["tags"]:
    issue_matched.update(tags)

In [37]:
print("final matched:",len(angular_matched.union(issue_matched)))

final matched: 9790


## Similarity Analysis

In [43]:
df_gitter = pd.read_csv(channel2+'/'+channel2+'_profile_tags.csv')

In [44]:
df_issue = pd.read_csv(channel2+'/'+channel2+'_issue_tags.csv')

In [45]:
df_github = pd.read_csv(channel2+'/'+channel2+'_user_meta.csv')

In [48]:
df_name = pd.read_json("../Data/Gitter_Channels/"+channel2+"/"+channel2+'_user.json')

In [65]:
Ids = []
for username in df_github["User"]:
    Ids.append(df_name[df_name["username"] == username]["id"].values[0])

In [67]:
df_github["user_Id"] = Ids

In [68]:
df_github.head()

,User,Repo_num,Repo_languages,Repo_topics,user_Id
0,007444,3,"['Ruby', 'Shell', 'HTML']",[],5f727524d73408ce4ff021fe
1,01CodeLT,3,"['TypeScript', 'HTML', 'SCSS', 'JavaScript', '...","['meeting', 'jw', 'bible', 'epub-reader', 'jeh...",5f86b560d73408ce4ff17d63
2,02arpit07,50,"['TypeScript', 'Julia', 'Go', 'PHP', 'SCSS', '...",[],5f2f0157d73408ce4febb81b
3,03williammoses,4,"['TeX', 'Java', 'Roff']",[],5e5c9d01d73408ce4fdb4239
4,04raghuramvugs,8,"['CSS', 'TypeScript', 'JavaScript', 'HTML']",[],5bbf743bd73408ce4fab237b


In [69]:
df_github.to_csv(channel2+'/'+channel2+'_user_meta_tags.csv',index = None)

In [106]:
df_angular_sorted.head(10)

,user_Id,count
0,57787d7fc2f0db084a2127f1,11700
1,5666e77c16b6c7089cbde7d5,7588
2,5724bc6d659847a7aff549ce,6538
3,576a94bdc2f0db084a1f5332,4644
4,5515c77715522ed4b3ddf13e,4381
5,55b409ef0fc9f982beaafb85,3974
6,54a8b22bdb8155e6700e57c1,3872
7,563a5bc416b6c7089cb97cfa,3033
8,579228bd40f3a6eec05c03aa,2521
9,54d28ce0db8155e6700f7486,2219


In [107]:
df_angular_sorted["user_Id"][0:10].values

array(['57787d7fc2f0db084a2127f1', '5666e77c16b6c7089cbde7d5',
       '5724bc6d659847a7aff549ce', '576a94bdc2f0db084a1f5332',
       '5515c77715522ed4b3ddf13e', '55b409ef0fc9f982beaafb85',
       '54a8b22bdb8155e6700e57c1', '563a5bc416b6c7089cb97cfa',
       '579228bd40f3a6eec05c03aa', '54d28ce0db8155e6700f7486'],
      dtype=object)